In [3]:
## testing printing output from console
import subprocess

cmd = [ 'echo', '"Welcome to my PySpark analysis of some StackExchange Data"' ]
output = subprocess.Popen( cmd, stdout=subprocess.PIPE ).communicate()[0]
print(output)

b'"Welcome to my PySpark analysis of some StackExchange Data"\n'


# Load Libraries

In [4]:
import gc #garbage collection
import time
import numpy as np
import pandas as pd
from datetime import datetime

# Choose Datasets

In [5]:
data_array = [
    "buddhism",
    "economics",
    "fitness",
    "health",
    "interpersonal"
]

# Download XML Data

In [6]:
## call download script - EDIT to feed in user data_array
#!bash 0-dataset-download.sh

# Convert to Parquet

In [7]:
## loop through data_array
#for i in data_array:
#    !python convert-xml-to-parquet.py "$i"

# Load PySpark

In [8]:
%run -i '1-load-pyspark.py'

The Spark UI, version 2.4.3, is available at: http://192.168.0.26:4040/ and the defaultParallelism is 4


# Load easyFunctions and Transformers

In [9]:
## easy functions
%run -i 'load_parquet_data.py'
%run -i 'export_parquet_data.py'
%run -i 'show_save_results.py'
%run -i 'show_spark_df.py'

## pipeline transformers
%run -i 'nltkWordPunctTokeniser.py'
%run -i 'nltkSenteniser.py'

# Load Initial or Clean Data

In [10]:
%%time
print(datetime.now().time())
datasets = load_parquet_data(data_array, kind='clean', printSchema=False)

10:33:36.832307
CPU times: user 4.72 ms, sys: 2.77 ms, total: 7.49 ms
Wall time: 4.35 s


In [11]:
## count total questions
s = 0
for i in data_array:
    s = s + datasets[i].count()
    print(f'{i}: {datasets[i].count()}')
s

buddhism: 5801
economics: 7870
fitness: 8225
health: 5617
interpersonal: 3123


30636

# Clean Data

In [12]:
%%time
print(datetime.now().time())
#%run -i '2-clean-datasets.py'

10:33:44.808520
CPU times: user 211 µs, sys: 67 µs, total: 278 µs
Wall time: 228 µs


# Counts and LDA Feature Engineering

In [13]:
%%time
# about 15 min with 10 iterations and params
print(datetime.now().time())
#%run -i '3-feat-engineering.py'
###
# TRY different values of alpha and rho
# TRY different values of K
# TRY 'em' optimizer instead of 'online'
# alpha=0.01, rho=0.01 : 

10:33:44.824529
CPU times: user 725 µs, sys: 324 µs, total: 1.05 ms
Wall time: 1.55 ms


# Export Clean Data

In [14]:
%%time
print(datetime.now().time())
#export_parquet_data(dataArray=data_array, datasetDict=datasets)

10:33:44.836203
CPU times: user 372 µs, sys: 114 µs, total: 486 µs
Wall time: 411 µs


# EDA

In [15]:
%%time
print(datetime.now().time())
#%run -i '4-final-eda.py'

10:33:44.849551
CPU times: user 374 µs, sys: 102 µs, total: 476 µs
Wall time: 412 µs


In [16]:
## correlations between score and viewcount
for i in data_array:
    s = i.title() + ' & ' + str(round( datasets[i].stat.corr("score", "viewcount"), 2 )) + ' \\\\'
    print(s)

Buddhism & 0.44 \\
Economics & 0.3 \\
Fitness & 0.36 \\
Health & 0.15 \\
Interpersonal & 0.86 \\


# Train/Test Splits

In [17]:
## garbage collector to speed up computation
collected = gc.collect()
print(f'Garbage collector: collected {collected} objects.')

Garbage collector: collected 70 objects.


In [65]:
%%time
print(datetime.now().time())
%run -i '6c-time-train-test-split-60.py'
#%run -i '6d-rand-train-test-split-60.py'

## check standard deviations of variables
print('\n')
for i in data_array:
    s = i.title() + ' & ' + str(round( pd.to_numeric(train[i].describe('score').select('score').toPandas().iloc[2][0]), 2 )) + ' & ' + \
    str(round( pd.to_numeric(test[i].describe('score').select('score').toPandas().iloc[2][0]), 2 )) + ' \\\\'
    print(s)

10:47:05.801994
economics: 0.4, from 3148 test and 7870 total
buddhism: 0.40062058265816236, from 2324 test and 5801 total
fitness: 0.4006079027355623, from 3295 test and 8225 total
health: 0.40092576108242833, from 2252 test and 5617 total
interpersonal: 0.398975344220301, from 1246 test and 3123 total


Buddhism & 3.98 & 2.07 \\
Economics & 3.69 & 2.7 \\
Fitness & 6.34 & 2.17 \\
Health & 4.85 & 2.21 \\
Interpersonal & 25.81 & 19.13 \\
CPU times: user 74.2 ms, sys: 22.8 ms, total: 97 ms
Wall time: 1.96 s


In [66]:
## garbage collector to speed up computation
collected = gc.collect()
print(f'Garbage collector: collected {collected} objects.')

Garbage collector: collected 1729 objects.


In [67]:
'''
interesting to see how skewed rus_stackoverflow posts are to more posts in recent years
'''

'\ninteresting to see how skewed rus_stackoverflow posts are to more posts in recent years\n'

# Create Results Dictionary

In [68]:
RESULTS = {}
for i in data_array:
    # capitalise keys
    RESULTS[i.title()] = {}

# Silly Mean Model

In [69]:
import pyspark.sql.functions as F

## choose target variable
target = 'score'

## create mean dictionaries
y_ravi_tr_means = {}

## calculate the mean of each forum, using ONLY training set
for i in data_array:
    y_ravi_tr_means[i] = train[i].select(target).rdd.flatMap(lambda x: x).mean()

## import rmse evaluator
from pyspark.ml.evaluation import RegressionEvaluator

## create dictionaries for training and testing (baseline) rmse 
base = {}
tr_rmse = {}

## modelling
for i in data_array:

    ## initial variable for timing
    t0 = time.time()
    
    ## train silly mean model by assigning training set mean for training and testing predictions
    train[i] = train[i].withColumn('mean_pred', F.lit(y_ravi_tr_means[i]))
    test[i] = test[i].withColumn('mean_pred', F.lit(y_ravi_tr_means[i]))

    ## evaluate silly mean model, on both training and testing set
    evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='mean_pred')
    tr_rmse[i] = round( evaluator.evaluate(train[i]), 2)
    base[i] = round( evaluator.evaluate(test[i]), 2)

    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m mean\033[0m model is {base[i]}")

    ## record time taken
    timet = round( time.time() - t0, 2 )
    
    ## store as dictionary inside RESULTS dictionary, initiating dataset name entries first
    RESULTS[i.title()]['0silly_mean.0tr_rmse'] = tr_rmse[i]
    RESULTS[i.title()]['0silly_mean.1rmse'] = base[i]
    RESULTS[i.title()]['0silly_mean.2timet'] = timet
    
## record results
show_save_results(RESULTS)

The root-mean-square error of buddhism's mean model is 3.44
The root-mean-square error of economics's mean model is 3.1
The root-mean-square error of fitness's mean model is 3.56
The root-mean-square error of health's mean model is 2.82
The root-mean-square error of interpersonal's mean model is 20.93


,0silly_mean.0tr_rmse,0silly_mean.1rmse,0silly_mean.2timet
Buddhism,3.98,3.44,0.44
Economics,3.69,3.10,0.32
Fitness,6.34,3.56,0.25
Health,4.85,2.82,0.27
Interpersonal,25.81,20.93,0.32


\begin{tabular}{lrrr}
\toprule
{} &  0silly\_mean.0tr\_rmse &  0silly\_mean.1rmse &  0silly\_mean.2timet \\
\midrule
Buddhism      &                  3.98 &               3.44 &                0.44 \\
Economics     &                  3.69 &               3.10 &                0.32 \\
Fitness       &                  6.34 &               3.56 &                0.25 \\
Health        &                  4.85 &               2.82 &                0.27 \\
Interpersonal &                 25.81 &              20.93 &                0.32 \\
\bottomrule
\end{tabular}



# Viewcount Model

In [ ]:
%%time
print(datetime.now().time())

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['viewcount']
datet_variables = ['clean_date']

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

'''## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression on just viewcount
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(#maxIter=100, # this doesn't change anything
                      #regParam=0.3, # using regularisation parameter here useless since there is one feature
                      #elasticNetParam=0.8,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='viewcount_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
#.addGrid(lr.regParam, [1e-3, 1.])
#.addGrid(lr.elasticNetParam, [1e-3, 1.])
paramGrid = ParamGridBuilder() \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='viewcount_pred')

## set up cross validation for parameter tuning
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    
    ## fitting on train and predicting on train/test
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
        
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m viewcount\033[0m model is {rmse}")

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['1viewcount.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['1viewcount.1rmse'] = rmse
    RESULTS[i.title()]['1viewcount.2imprv'] = impr
    RESULTS[i.title()]['1viewcount.3timet'] = timet
    
## record results
show_save_results(RESULTS)

10:47:27.710090
The root-mean-square error of buddhism's viewcount model is 3.02


In [ ]:
'''
Interesting that there are different improvements of viewcount over mean-only prediciton
'''

In [ ]:
## garbage collector to speed up computation
collected = gc.collect()
print(f'Garbage collector: collected {collected} objects.')

## Count Model

In [ ]:
%%time
print(datetime.now().time())

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['body_word_cnt', 'titl_word_cnt', 'body_char_cnt', 
                   'titl_char_cnt', 'body_sent_cnt', 'titl_sent_cnt']
datet_variables = ['clean_date']

'''## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

## NUMERICAL columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## create PROCESSING pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(maxIter=100,
                      regParam=1,
                      elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='counts_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-3, 1.]) \
    .addGrid(lr.regParam, [1e-3, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='counts_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m counts\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2counts.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2counts.1rmse'] = rmse
    RESULTS[i.title()]['2counts.2imprv'] = impr
    RESULTS[i.title()]['2counts.3timet'] = timet
    RESULTS[i.title()]['2counts.4elastic'] = ela_param
    RESULTS[i.title()]['2counts.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

## Text Model

In [ ]:
%%time
print(datetime.now().time())
# ENGLISH
# 8min 56s for no CV and no GRIDSEARCH
# 17min 10s for 3-CV and no GRIDSEARCH
# SMALL DATASETS
# 36min 55s for no CV and no GRIDSEARCH
# 12min 35s for 3-CV and no GRIDSEARCH
# 8min 32s for 2-CV and no GRIDSEARCH

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
textt_variables = ['title', 'clean_body']
datet_variables = ['clean_date']

'''## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

## textual columns
# tokenising text cols with custom transformer
nltk_tokeniser_body = nltkWordPunctTokeniser(
    inputCol='clean_body', outputCol='body_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

nltk_tokeniser_title = nltkWordPunctTokeniser(
    inputCol='title', outputCol='titl_words',  
    stopwords=set(nltk.corpus.stopwords.words('english')))

# count occurence of tokens, i.e. create dfm
cnt_vectrizr_body = CountVectorizer(inputCol='body_words', outputCol='body_raw_feats') #!!! minDF???
cnt_vectrizr_title = CountVectorizer(inputCol='titl_words', outputCol='titl_raw_feats')

# create IDF dfm
idf_body = IDF(inputCol="body_raw_feats", outputCol="body_feats")
idf_title = IDF(inputCol="titl_raw_feats", outputCol="titl_feats")

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['body_feats', 'titl_feats'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    nltk_tokeniser_body, 
    nltk_tokeniser_title,
    cnt_vectrizr_body,
    cnt_vectrizr_title,
    idf_body,
    idf_title,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(maxIter=100,
                      regParam=1,
                      elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='tokens_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-3, 1.]) \
    .addGrid(lr.regParam, [1e-3, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='tokens_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m tokens\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2tokens.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2tokens.1rmse'] = rmse
    RESULTS[i.title()]['2tokens.2imprv'] = impr
    RESULTS[i.title()]['2tokens.3timet'] = timet
    RESULTS[i.title()]['2tokens.4elastic'] = ela_param
    RESULTS[i.title()]['2tokens.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

In [ ]:
## check predictions aren't constant
models['health'].transform(test['health']).select('tokens_pred').take(10)

In [ ]:
"""why the heck does everything besides interpersonal have constant predictions - it's not the parameters or the size of the data"""
"""it's the number of questions in the data"""

# LDA Features Model

In [ ]:
%%time
print(datetime.now().time())

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['btd[0]', 'btd[1]', 'btd[2]', 'btd[3]', 'btd[4]', 'btd[5]', 'btd[6]', 'btd[7]', 'btd[8]', 
                   'btd[9]', 'ttd[0]', 'ttd[1]', 'ttd[2]', 'ttd[3]', 'ttd[4]', 'ttd[5]', 'ttd[6]', 'ttd[7]', 
                   'ttd[8]', 'ttd[9]']
datet_variables = ['clean_date']

'''## date columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')'''

## numerical columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])

## create processing pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[numic_pipeline, process_assembler])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(maxIter=100,
                      regParam=1,
                      elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='topics_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
    
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-3, 1.]) \
    .addGrid(lr.regParam, [1e-3, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='topics_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m topics\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['2topics.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['2topics.1rmse'] = rmse
    RESULTS[i.title()]['2topics.2imprv'] = impr
    RESULTS[i.title()]['2topics.3timet'] = timet
    RESULTS[i.title()]['2topics.4elastic'] = ela_param
    RESULTS[i.title()]['2topics.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

# Final Model

In [39]:
%%time
print(datetime.now().time())

from pyspark.ml.pipeline import Pipeline
from pyspark.ml.feature import CountVectorizer, IDF, StandardScaler, VectorAssembler, VectorSlicer

########################
##### CHOOSE FEATS ##### can't get date right
########################

## define features to predict on
target = 'score'
numic_variables = ['btd[0]', 'btd[1]', 'btd[2]', 'btd[3]', 'btd[4]', 'btd[5]', 'btd[6]', 'btd[7]', 'btd[8]', 
                   'btd[9]', 'ttd[0]', 'ttd[1]', 'ttd[2]', 'ttd[3]', 'ttd[4]', 'ttd[5]', 'ttd[6]', 'ttd[7]', 
                   'ttd[8]', 'ttd[9]', 'body_word_cnt', 'titl_word_cnt', 'body_char_cnt', 
                   'titl_char_cnt', 'body_sent_cnt', 'titl_sent_cnt']
datet_variables = ['clean_date']

### DATE columns
datet_assembler = VectorAssembler(inputCols=datet_variables, outputCol='datet_data')

### NUMERICAL columns
numic_assembler = VectorAssembler(inputCols=numic_variables, outputCol='numic_data') # have to put in single col
standardiser = StandardScaler(inputCol='numic_data', outputCol='numic_data_std')    
numic_pipeline = Pipeline(stages=[numic_assembler, standardiser])


## create PROCESSING pipeline
process_assembler = VectorAssembler(inputCols=['numic_data'], #inputCols=['datet_data']
                                    outputCol='features') 
process_pipeline = Pipeline(stages=[  #inputCols=['datet_data']
    numic_pipeline,
    process_assembler
])

########################
##### CHOOSE MODEL #####
########################

## linear regression model
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
    
lr = LinearRegression(maxIter=100,
                      regParam=1,
                      elasticNetParam=1,
                      featuresCol='features',
                      labelCol=target,
                      predictionCol='finalm_pred')

## make final pipeline
final_pipeline = Pipeline(stages=[process_pipeline, lr])

## import methods for tuning
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

## set up grid for parameter tuning: 
'''NEEDED, BUT IMMENSELY SLOWING DOWN'''
# Ravi et al use L2, aka ridge, aka elasticNetParam=0
# regParam is the value of lambda
    
paramGrid = ParamGridBuilder() \
    .addGrid(lr.elasticNetParam, [1e-3, 1.]) \
    .addGrid(lr.regParam, [1e-3, 1.]) \
    .build()

## set up rmse evaluator
evaluator = RegressionEvaluator(metricName='rmse', labelCol=target, predictionCol='finalm_pred')

## set up cross validation for parameter tuning
'''DEFINITELY SLOWING DOWN'''
crossval = CrossValidator(estimator=final_pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2)
## create models dict
models = {}

## modelling
for i in data_array:
    
    ## initial variable for timing
    t0 = time.time()
    
    ## fit on training set with CV
    cvmodel = crossval.fit(train[i])
    models[i] = cvmodel
    
    ## predict and evaluate
    tr_rmse = round( evaluator.evaluate(cvmodel.transform(train[i])), 2 )
    rmse = round( evaluator.evaluate(cvmodel.transform(test[i])), 2 )
    print(f"The root-mean-square error of \033[94m{i}'s\033[0m\033[92m final\033[0m model is {rmse}")
    
    ## get params
    # elasticnet
    ela_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[1]
    ela_param = cvmodel.bestModel.stages[-1].extractParamMap()[ela_key]
    # reg'sation
    reg_key = list(cvmodel.bestModel.stages[-1].extractParamMap().keys())[9]
    reg_param = cvmodel.bestModel.stages[-1].extractParamMap()[reg_key]

    ## calculate improvement over median baseline
    impr = round( (rmse/base[i] - 1)*-100, 2 )
    
    ## record time taken
    timet = round( time.time() - t0, 2 )

    ## store as dictionary inside RESULTS dictionary
    RESULTS[i.title()]['3final.0tr_rmse'] = tr_rmse
    RESULTS[i.title()]['3final.1rmse'] = rmse
    RESULTS[i.title()]['3final.2imprv'] = impr
    RESULTS[i.title()]['3final.3timet'] = timet
    RESULTS[i.title()]['3final.4elastic'] = ela_param
    RESULTS[i.title()]['3final.5regular'] = reg_param
    
## record results
show_save_results(RESULTS)

17:53:37.647900
The root-mean-square error of buddhism's final model is 3.52
The root-mean-square error of economics's final model is 3.6
The root-mean-square error of fitness's final model is 5.17
The root-mean-square error of health's final model is 3.68
The root-mean-square error of interpersonal's final model is 28.13


,3final.0tr_rmse,3final.1rmse,3final.2imprv,3final.3timet,3final.4elastic,3final.5regular
Buddhism,3.62,3.52,0.56,10.08,0.001,1.000
Economics,3.24,3.60,0.83,7.91,0.001,1.000
Fitness,5.29,5.17,0.39,8.84,1.000,0.001
Health,4.30,3.68,0.81,6.97,0.001,1.000
Interpersonal,20.12,28.13,0.35,7.03,1.000,1.000


\begin{tabular}{lrrrrrr}
\toprule
{} &  3final.0tr\_rmse &  3final.1rmse &  3final.2imprv &  3final.3timet &  3final.4elastic &  3final.5regular \\
\midrule
Buddhism      &             3.62 &          3.52 &           0.56 &          10.08 &            0.001 &            1.000 \\
Economics     &             3.24 &          3.60 &           0.83 &           7.91 &            0.001 &            1.000 \\
Fitness       &             5.29 &          5.17 &           0.39 &           8.84 &            1.000 &            0.001 \\
Health        &             4.30 &          3.68 &           0.81 &           6.97 &            0.001 &            1.000 \\
Interpersonal &            20.12 &         28.13 &           0.35 &           7.03 &            1.000 &            1.000 \\
\bottomrule
\end{tabular}

CPU times: user 4.32 s, sys: 1.34 s, total: 5.66 s
Wall time: 40.9 s


In [42]:
## check predictions aren't constant
models['health'].transform(test['health']).select('finalm_pred').take(100)

[Row(finalm_pred=2.9190986652103756),
 Row(finalm_pred=2.4109049599660364),
 Row(finalm_pred=3.158419654331349),
 Row(finalm_pred=2.9567919243386123),
 Row(finalm_pred=3.022591392683038),
 Row(finalm_pred=3.6032996635176935),
 Row(finalm_pred=3.2730229802480872),
 Row(finalm_pred=4.19333008489856),
 Row(finalm_pred=3.593894033650666),
 Row(finalm_pred=2.9712486943446104),
 Row(finalm_pred=3.007316082996332),
 Row(finalm_pred=2.83153581489662),
 Row(finalm_pred=2.6636758750639578),
 Row(finalm_pred=3.0695896812849663),
 Row(finalm_pred=3.456308669254049),
 Row(finalm_pred=3.8776198420716765),
 Row(finalm_pred=3.7269295145918573),
 Row(finalm_pred=3.6364471263597076),
 Row(finalm_pred=3.094029818129936),
 Row(finalm_pred=3.6653403486524465),
 Row(finalm_pred=3.7780235040431887),
 Row(finalm_pred=3.8600674178571555),
 Row(finalm_pred=3.3346961603972622),
 Row(finalm_pred=3.441695020198778),
 Row(finalm_pred=3.5848764123695145),
 Row(finalm_pred=3.494866165355581),
 Row(finalm_pred=3.10997

In [ ]:
a

# Save predictions

In [ ]:
(trained_pipeline
 .transform(datasets['english'])
 .select(
    indep_text_variables + ["prediction"]
 )
 .write
 .parquet("linreg_prediction.parquet")
)

In [ ]:
linreg_predictions = spark.read.parquet("linreg_prediction.parquet")

In [ ]:
linreg_predictions.toPandas().head()

In [ ]:
linreg_predictions.select("prediction").describe().toPandas()

# Save pipelines

In [ ]:
from joblib import dump, load
dump(estimator_pipeline, 'pipeline.joblib') 

reloaded = load("pipeline.joblib")

#Now we can predict directly!

reloaded.predict(X)[:10]

In [ ]:
## save models DOESN'T WORK BECAUSE: 'NLTKWordPunctTokenizer' object has no attribute '_to_java'
for i in data_array:
    param_dict[i].save(f'{i}-pipeline') 

# Convert notebook to python file

In [ ]:
!jupyter nbconvert --to script 0-master-notebook-pipelines.ipynb